In [1]:
import os, sagemaker, subprocess, boto3
from datetime import datetime
from sagemaker import s3
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from sagemaker.serverless import ServerlessInferenceConfig
from sagemaker.deserializers import JSONDeserializer

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
!pip3 install ultralytics

In [3]:
from ultralytics import YOLO

In [4]:
## Choose a model:
model_name = './code/best.pt'

YOLO(model_name)
# os.system(f'mv {model_name} code/.')

bashCommand = "tar -cpzf  model.tar.gz code/"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

## 1.3 Zip the code and model into `model.tar.gz` and upload it to specific S3 bucket
Here permission is granted to the S3 bucket by the IAM role

In [5]:
s3_client = boto3.client('s3')
bucket = 's3://adas-deployment-bucket'
print(f'Bucket: {bucket}')
sess = sagemaker.Session(default_bucket=bucket.split('s3://')[-1])

prefix = "yolov8/images-endpoint"

Bucket: s3://adas-deployment-bucket


In [6]:
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

role = get_execution_role()
print(f'Role: {role}')

model_data = s3.S3Uploader.upload("model.tar.gz", bucket + "/" + prefix)
print(f'Model Data: {model_data}')

Role: arn:aws:iam::141178960309:role/service-role/SageMaker-s3-admin
Model Data: s3://adas-deployment-bucket/yolov8/images-endpoint/model.tar.gz


## 1.4 Create the SageMaker PyTorchModel

In [7]:
model = PyTorchModel(entry_point='inference.py',
                     model_data=model_data, 
                     framework_version='1.12', 
                     py_version='py38',
                     role=role,
                     env={'TS_MAX_RESPONSE_SIZE':'20000000', 'YOLOV8_MODEL': model_name},
                     sagemaker_session=sess)

## 1.5 Deploy the model on SageMaker Endpoint:

In [8]:
# Specify MemorySizeInMB and MaxConcurrency in the serverless config object
serverless_config = ServerlessInferenceConfig(
  memory_size_in_mb=3072,
  max_concurrency=1,
)

In [ ]:
INSTANCE_TYPE = 'ml.t2.medium'
ENDPOINT_NAME = 'yolov8-images-serverless-endpoint' 

# Store the endpoint name in the history to be accessed by 2_TestEndpoint.ipynb notebook
%store ENDPOINT_NAME
print(f'Endpoint Name: {ENDPOINT_NAME}')

# Deploys the model to a SageMaker serverless endpoint
serverless_predictor = model.deploy(serverless_inference_config=serverless_config,
                                    initial_instance_count=1, 
                                    instance_type=INSTANCE_TYPE,
                                    deserializer=JSONDeserializer(),
                                    endpoint_name=ENDPOINT_NAME
                                    )

Stored 'ENDPOINT_NAME' (str)
Endpoint Name: yolov8-images-serverless-endpoint
---